## Chat With SQL

First get any .db file here i have Chinook.db file

In [1]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())

sqlite
['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'invoices', 'media_types', 'playlist_track', 'playlists', 'tracks']


In [2]:
db.run("SELECT * FROM artists LIMIT 10;")

"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

Chains are compositions of LangChain Runnables which support applications whose steps are predictable. We can create a simple chain that takes a question and does the following.
* Convert the question into a SQL query
* Execute the query
* Use the result to answer the original question

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

llm = ChatGroq(model="llama3-8b-8192")

In [70]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke(
    {"question": "How many employees are there?"}
)
response

'Question: How many employees are there?\nSQLQuery: SELECT COUNT(*) FROM employees;'

In [52]:
db.run(response)

'[(8,)]'

In [53]:
test_query = llm.invoke("Just generate a SQL query to get the number of employees in employees table. just return the sql query")

In [54]:
db.run("SELECT COUNT(*) FROM employees;")

'[(8,)]'

In [55]:
db.run(test_query.content)

'[(8,)]'

In [56]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)

chain = write_query | execute_query
chain.invoke({
    "question": "How many employees are there? Just response with the query like this: SELECT * From table; and nothing else other than query."
})

'[(8,)]'

In [57]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [85]:
# now answering the question with the sql query result as well.

from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """
    Given the following user question, corresponding SQL query and the result, answer the user question.
    
    Question: {question}
    query : {query}
    SQL Result: {result}
    Answer:
    """
)

chain = (
    RunnablePassthrough.assign(query=write_query).assign(result=itemgetter("query") | execute_query)

    | answer_prompt
    | llm
    | StrOutputParser()
)

In [100]:
# sahi sanga li raakheko chaina yesle


chain.invoke({
    "question": "can you say 10 artists? Just response with the query in sql and nothing else other than query."
})

'SELECT "Name" FROM artists LIMIT 10'

In [101]:
chain.invoke({
    "question": "How many artists are there? Just response with the query in sql and nothing else other than query."
})

'There are 275 artists.'

In [103]:
chain.invoke(
    {
        "question": "How many employees are there?"
    }
)

'The user question is: How many employees are there?\n\nHowever, the SQL query provided is not valid. It seems that the SQL query is trying to execute the question itself, which is not a valid SQL syntax.\n\nThe correct SQL query to answer the question "How many employees are there?" would be:\n\nSELECT COUNT(*) FROM employees;\n\nThis query will count the number of rows in the "employees" table and return the result.\n\nThe correct answer to the user question would be the result of this query, which is the number of employees in the "employees" table.'

In [61]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

## Agents

In [104]:
# using sqltool agents for robustness and error less response

from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()

tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001C5E9A7A090>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001C5E9A7A090>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001C5E9A7A090>),
 QuerySQLCheckerTool(description='Use this tool to 

In [105]:
from langchain_core.messages import SystemMessage

SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""

system_message = SystemMessage(
    content=SQL_PREFIX
)

In [106]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(
    llm,
    tools,
    messages_modifier=system_message
)

In [110]:
agent_executor.invoke(
    {
        "messages": HumanMessage(content="Hello who are you?")
    }
)

{'messages': [HumanMessage(content='Hello who are you?', id='d831f81f-3870-4c74-9875-fd3a63feb9ab'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_kn07', 'function': {'arguments': '{"tool_input":""}', 'name': 'sql_db_list_tables'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 1630, 'total_tokens': 1704, 'completion_time': 0.05804546, 'prompt_time': 0.246238681, 'queue_time': None, 'total_time': 0.304284141}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_af05557ca2', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-cf10b807-90dd-425a-b971-b246898b7420-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {'tool_input': ''}, 'id': 'call_kn07'}]),
  ToolMessage(content='albums, artists, customers, employees, genres, invoice_items, invoices, media_types, playlist_track, playlists, tracks', name='sql_db_list_tables', id='c7055021-cd76-426c-b47b-29d8217250d7', tool_call_id='call_kn07'),
  AI

In [107]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="Which country's customers spent the most?")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_td3k', 'function': {'arguments': '{}', 'name': 'sql_db_list_tables'}, 'type': 'function'}, {'id': 'call_9s82', 'function': {'arguments': '{"table_names":"customers"}', 'name': 'sql_db_schema'}, 'type': 'function'}, {'id': 'call_bxr5', 'function': {'arguments': '{"query":"SELECT country, SUM(total_spent) AS total_spent FROM customers GROUP BY country ORDER BY total_spent DESC LIMIT 1"}', 'name': 'sql_db_query'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 200, 'prompt_tokens': 1633, 'total_tokens': 1833, 'completion_time': 0.158688629, 'prompt_time': 0.287298188, 'queue_time': None, 'total_time': 0.445986817}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-742bbce9-8745-48e9-bd6c-116a075f98bd-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {}, 'id': 'call_td3k'}, {'name': '

In [111]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="Describe the playlisttrack table")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_fs5y', 'function': {'arguments': '{"tool_input":""}', 'name': 'sql_db_list_tables'}, 'type': 'function'}, {'id': 'call_a49c', 'function': {'arguments': '{"table_names":"playlisttrack"}', 'name': 'sql_db_schema'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 117, 'prompt_tokens': 1630, 'total_tokens': 1747, 'completion_time': 0.091777212, 'prompt_time': 0.242953184, 'queue_time': None, 'total_time': 0.334730396}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_c4a72fb330', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-38325418-ad49-4a0a-8fd8-4e9de8f3393c-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {'tool_input': ''}, 'id': 'call_fs5y'}, {'name': 'sql_db_schema', 'args': {'table_names': 'playlisttrack'}, 'id': 'call_a49c'}])]}}
----
{'tools': {'messages': [ToolMessage(content='albums, artists, customers, employees, genres, invoice

explore more about `**Retriever tool**` to use here